# Reconocimiento Facial usando Random Forest

Se entrena el clasificador Random Forest para reconocer imágenes de una base de datos de imágenes de video. 
Se usa el detector de rostros haarcascades en la librería de OpenCV para reconocer rostro en real time.
Los resultados son excelentes. 

In [4]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [5]:
# Versiones de los paquetes usados en este notebook

%reload_ext watermark
%watermark -a "César S. Hooper/ Dic 2021" --iversions
!python --version

Author: César S. Hooper/ Dic 2021

matplotlib: 3.3.4
seaborn   : 0.11.1
sklearn   : 0.24.1
cv2       : 4.5.4-dev
numpy     : 1.19.5
skimage   : 0.18.1
pandas    : 1.2.4

Python 3.8.8


Para el entrenamiento usamos las imágenes con las que entrenamos un modelo de reconocimiento facial con OpenCV.
Dejo el link
https://github.com/hoopercesar/Reconocimiento_facial/blob/main/Reconocimiento_Facial_con_OpenCV.ipynb

Estas imágenes fueron extraídas de dos registros de video, se guardaron en formato jpg con tamaño 100x100 pixeles con los tres canales BGR. 

### Preparración de los datos

In [6]:
# Cargamos las imágenes 321 imágenes de cada archivo, las transformamos al gris 
# y las guardamos en una lista como array de 100x100. 
# En total serán 642 matrices de con datos en formato de imagen dtype='uint8'

directorios = ['cesar', 'fernanda']
path = 'C:/Users/Cesar Hooper/Desktop/github/reconocimiento_facial/data/'
lista = []
for directorio in directorios:
    counter = 0
    persona = path + directorio
    for foto in os.listdir(persona):
        
        while counter<=320:
            img = cv2.imread(persona + '/' + foto)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            lista.append(img)
            counter += 1
# tranformar la lista en array
x = np.array(lista)

In [7]:
# cada matriz (100x100) se transforma en un un array de 1 fila por 10000 columnas
# por tanto, x será un array de 642 vectores de 10000 columnas

x = x.reshape(-1,10000) # este es un array de (642, 10000)

# creamos diccionario y luego dataframe
s = {}
for columna in range(x.shape[1]):
    s['p'+str(columna)] = list(x[:,columna])

dataframe = pd.DataFrame(s)    

# crear labels
k = np.arange(2)
labels = np.repeat(k, dataframe.shape[0]/2)

dataframe['target'] = labels

# randomizamos las filas del dataframe
dataframe_random = dataframe.sample(frac=1).reset_index(drop=True)

y = dataframe_random['target']
X = dataframe_random.drop(['target'], axis=1)

### Creación y entrenamiento del modelo

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8) # 70% training and 30% test

#Crea el Classifier
clf=RandomForestClassifier(n_estimators=200)

# Entrena el modelo y luego usa y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print("Accuracy:%.5f"%metrics.accuracy_score(y_test, y_pred))

Accuracy:1.00000


In [21]:
cap = cv2.VideoCapture(0)
faceclassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
i = 0
while True:
    ret, frame = cap.read()
    
    if ret == False:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxframe = gray.copy()
    
    faces = faceclassif.detectMultiScale(gray, 1.3, 5)

    for faz in faces:
        x,y,h,w = faz
        rostro = auxframe[y:y+h, x:x+w]
        rostro = cv2.resize(rostro, (100,100), interpolation=cv2.INTER_CUBIC)
        
        z = rostro.reshape(-1, 10000)
        dataframe = pd.DataFrame(z)
            
       
        resultado = clf.predict(dataframe)
        prob = clf.predict_proba(dataframe)
        #print(prob)
        
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0),2)
                
        if resultado == 0:
            cv2.putText(frame, '{}'.format('Cesar ,' + str(prob[0][0])), (x,y-5), 1,1.3,(255,255,0),1,cv2.LINE_AA)
        elif resultado == 1:
            cv2.putText(frame, '{}'.format('Fer'), (x,y-5), 1,1.3,(255,255,0),1,cv2.LINE_AA)
        else:
            cv2.putText(frame, '{}'.format('desconocido'), (x,y-5), 1,1.3,(255,255,0),1,cv2.LINE_AA)
        
    
    cv2.imshow('frame', frame)
    
    k = cv2.waitKey(1)
    if k == 27: break

cap.release()
cv2.destroyAllWindows()

### Comentarios

El modelo reconoce relativamente bien los rostros, con alrededor de un 70% de certeza. 
El modelaje de imágenes y el entrenamiento de modelos de reconocimiento depende sensiblemente de la luz
y el contraste de las imágenes, por tanto, la capacidad del modelo, para reconocer una imagen, depende 
de estos parámetros. 
Es recomendable usar más imágenes para el entrenamiento. Se usaron 640 imagenes en este notebook, lo que es poco.
Pero este es sólo un estudio preliminar.

# FIN